In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split



In [2]:
# Directorios de las imágenes
path_good = "/content/drive/MyDrive/VISION POR COMPUTADOR/SEMANA 13/Lemon_quality/Lemon_quality/good_quality"
path_bad = "/content/drive/MyDrive/VISION POR COMPUTADOR/SEMANA 13/Lemon_quality/Lemon_quality/bad_quality"

In [3]:
# Cargar imágenes y etiquetas
images = []
labels = []

def load_images(path, label):
    for image_name in os.listdir(path):
        img_path = os.path.join(path, image_name)
        image = cv2.imread(img_path)
        image = cv2.resize(image, (100, 100))
        images.append(image)
        labels.append(label)

# Cargar imágenes de buena y mala calidad
load_images(path_good, 1)  # 1 para 'good_quality'
load_images(path_bad, 0)   # 0 para 'bad_quality'

# Convertir a un arreglo numpy
images = np.array(images)
labels = np.array(labels)

In [4]:
# Normalizamos las imágenes
images = images.astype('float32') / 255.0

# Dividimos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [5]:
from skimage.feature import hog
from skimage import exposure

# Función para extraer características HOG de cada imagen
def extract_hog_features(images):
    hog_features = []
    for img in images:
        # Convertir a escala de grises
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Extraer HOG
        features, hog_image = hog(gray, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=True)
        # Normalizar la imagen HOG
        features = exposure.rescale_intensity(hog_image, in_range=(0, 10))
        hog_features.append(features.flatten())
    return np.array(hog_features)

# Extraemos características HOG para los conjuntos de entrenamiento y prueba
X_train_hog = extract_hog_features(X_train)
X_test_hog = extract_hog_features(X_test)

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Crear y entrenar el modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_hog, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test_hog)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9354
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.92      0.93       190
           1       0.94      0.95      0.94       228

    accuracy                           0.94       418
   macro avg       0.94      0.93      0.93       418
weighted avg       0.94      0.94      0.94       418



In [7]:
from sklearn.model_selection import GridSearchCV

# Definir el rango de parámetros
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, None]
}

# GridSearchCV para optimizar el modelo
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train_hog, y_train)

# Mostrar los mejores parámetros encontrados
print("Best parameters found: ", grid_search.best_params_)

# Evaluar el modelo optimizado
best_model = grid_search.best_estimator_
y_pred_optimized = best_model.predict(X_test_hog)

# Evaluar el modelo optimizado
accuracy_optimized = accuracy_score(y_test, y_pred_optimized)
print(f"Optimized Accuracy: {accuracy_optimized:.4f}")
print("Optimized Classification Report:")
print(classification_report(y_test, y_pred_optimized))


Best parameters found:  {'max_depth': 10, 'n_estimators': 100}
Optimized Accuracy: 0.9258
Optimized Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.91      0.92       190
           1       0.93      0.94      0.93       228

    accuracy                           0.93       418
   macro avg       0.93      0.92      0.93       418
weighted avg       0.93      0.93      0.93       418

